In [2]:
import time
import os
import cv2

from lib.file.TIF import *
from lib.analysis.ROI_identification import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.file.TSV import RegionExporter
from lib.analysis.align import ImageAlign
from lib.file.DAT import TracesDAT
from lib.trace import Tracer
from lib.utilities import *
from lib.analysis.laminar_dist import *

In [10]:
# take every 'precursor' ROI file (listing the corners of a rectangular ROI) 
#    and generate a corresponding filled-in ROI file
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
enable_photoZ_interaction = False
# sets of files in each slice directory -- naming convention
precursor_file_token = '_roi_precursors' # e.g. slice3_roi_precursors.dat

# output files
rois_file_prefix = '_rois' # e.g. slice3_rois.dat

n_plots = 5
all_new_roi_files = []
for subdir, dirs, files in os.walk(data_dir):
    
    if 'notUsable' in subdir:
        continue
    if 'selected_zda' not in subdir:
        continue

    subdir += '/'

    for filename in files:
        fn = filename.split(".")[0]
        if fn.endswith(precursor_file_token):
            
            la_file = subdir + filename

            lines = None
            if os.path.exists(la_file):
                print("\tProcessing", la_file)

                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                    
                num_roi_precursors = int((len(lines)-1) / 7)
                print("\tnum_roi_precursors:", num_roi_precursors)
                all_rois = []
                for i in range(num_roi_precursors):
                    start_line_no = 7*i+4
                
                    corners = [int(x) for x in lines[start_line_no:start_line_no+4]] # each 4 lines are diode numbers of corners
                    print("\t\tcorners:", corners)
                    layer_axes = LayerAxes(corners)
                    laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                    print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                          laminar_axis_2.get_unit_vector())

                    # make our own ROIs from the list of edges and write them to a .dat file
                    roi_cr = ROICreator(layer_axes, roi_width=27, stim_point_spacer=False)
                    rois = roi_cr.get_rois()  # list of LaminarROI objects
                    all_rois += rois

                # write these ROIs to file
                new_filename = filename.replace(precursor_file_token, rois_file_prefix)
                roi_cr.rois = all_rois
                roi_cr.n_rois_created = len(all_rois)
                written_filename = roi_cr.write_roi_file(subdir, new_filename)
                all_new_roi_files.append(written_filename)

print("\nFiles written:\n", len(all_new_roi_files))

	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/slice2_roi_precursors.dat
	num_roi_precursors: 1
		corners: [674, 623, 6357, 6383]
edge_pts [[37, 79], [63, 79]] axis_pts [[34, 8], [63, 7]]
laminar axis unit vectors: [0.042215852683817515, 0.9991085135170146] [0.0, 1.0]
	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-10-23\selected_zda/slice4_roi_precursors.dat
	num_roi_precursors: 1
		corners: [1253, 2552, 6344, 4640]
edge_pts [[24, 79], [0, 58]] axis_pts [[53, 15], [72, 31]]
laminar axis unit vectors: [-0.7071067811865476, 0.7071067811865476] [-0.7765619863716894, 0.630040856867597]
	Processing C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/slice2_roi_precursors.dat
	num_roi_precursors: 1
		corners: [5785, 5497, 9, 44]
edge_pts [[9, 0], [44, 0]] axis_pts [[25, 72], [57, 68]]
laminar axis unit vectors: [-0.21693045781865616, -0.9761870601839528] [-0.1877758073012499, -0.9822119151142303]
	Proce